In [ ]:
!pip install line-bot-sdk flask

In [ ]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("line_secret_key",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
# 學你說話
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    print(event.message.text)
    
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))

In [ ]:
# # 把上面的回復改成處理過的內容


# @handler.add(MessageEvent, message=TextMessage)
# def handle_message(event):
    
#     print(event.message.text)
    
#     my_str = '你好，我是機器人，您說了 {} ，但我不明白您的意思...'.format(event.message.text)
    
#     line_bot_api.reply_message(
#         event.reply_token,
#         TextSendMessage(text=my_str))

In [ ]:
'''

啟動Server

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')